# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This project is a classification supervised learning problem because the variable to predict, i.e. if a student graduates or fails to graduate, is categorical. On this case this a dichotomous categorical variable where the only two possible values are "pass" or "fail".

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [39]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns) -1


# TODO: Calculate passing students
n_passed = len(student_data[student_data.passed == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data.passed == 'no'])

# TODO: Calculate graduation rate
grad_rate = (float(n_passed) / n_students)*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [12]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import  train_test_split
# TODO: Set the number of training points
num_train = 300


# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=42)


# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

Decision Trees:  
 - Main strengths are:
  - Simple to understand and to interpret. Uses a white box model where the results are easily explained by boolean logic. Trees can be visualised.
  - Requires little data preparation. Other techniques often require data normalisation, dummy variables need to be created and blank values to be removed. 
  - Data prediction cost is logarithmic in the number of data points used to train the tree.
  - Able to handle both numerical and categorical data. 
  - Able to handle multi-output problems.


- Main weaknesses are:
 - Overfitting due to over-complex trees that do not generalise the data well. This problem can be counteracted by pruning, setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree.
 - Unstability because small variations in the data might result in completely different trees. This problem can be counteracted by using decision trees within an ensemble.
 - Not-guarantee to return the globally optimal decision tree because this is a NP-complete problem. Therefore practical algorithms are based on heuristic methods such as locally optimal decisions made at each node. This problem can be counteracted by using decision trees within an ensemble, where the features and samples are randomly sampled with replacement.
 - Biased trees if some classes dominate. This problem can be counteracted by balancing dataset prior to fitting.


- This model is a good candidate for this problem for the following reasons:
 - This is simple to understand and interpret using visualizations and so it is a good first step to start exploring
 - The above weaknesses can be counteracted: 
     - In our dataset students who passes dominates against those failing. In order to avoid obtaining biased trees because of it we can balance the dataset first.   
     - This is a typical classification problem not involving hard to express problems like XOR, parity or multiplexing.
 - Instead of using it alone, due to lack of stability, I would rather try to use it in an ensemble learner by training multiple trees where the features and samples are randomly sampled with replacement. 
 
Ensemble methods: 

- Main strengths are:
    - The main one is that it improves generalizability / robustness over a single estimator by combining the predictions of several base estimators.
    
    Two of the original techniques are bagging (subsampling the training data) and boosting (focusing on misclassified training instances and re-learning with different weights):
    - For baggging the main strength is to reduce the instability (where slightly different inputs leads to very different outputs) of the underlying learning algorithms. The combined estimator is usually better than any of the single base estimator because its variance is reduced.  
    - For boosting the main strengths are 
        - Greater capability to classify correctly "hard instances" by reweighting examples near the margin (positive and negative examples) and penalising classifiers that have a poor accuracy.
        - You may use many different algorithms as base learners

- Main weaknesses are:
     - Increased storage a a direct consequence of the requirement that all component classifiers, instead of a single classifier, need to be stored after training. The total storage depends on the size of each component classifier itself and the size of the ensemble (number of classifiers in the ensemble). 
     - Increased computation to classify an input query, all component classifiers (instead of a single classifier) must be processed, and thus it requires more execution time. 
     - Decreased comprehensibility due to involvement of multiple classifiers in decision-making, it is more difficult for users to perceive the underlying reasoning process leading to a decision
     
- This model is a good candidate for this problem because it has the advantages of decision trees (see above) and counteract its weaknesses by controlling overfitting and instability. One expected benefit of obtaining a robust model for this case, where a very reduced student sample (n = 300) has been drawn from the population of origin of all students, is that we can be more confident that we would have obtained similar models if they would have been trained with other student samples drawn from the same population.


Suppor Vector Machines
- Main strenghts are: 
    - Effective in high dimensional spaces.
    - Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
    - Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.
- Main weaknesses are:
    - The fit time complexity is more than quadratic with the number of samples which makes it hard to scale to dataset with more than a couple of 10000 samples.
    
- This model is good candite for this problem because it has a high dimensional space with 31 features and a reduced sample dataset. In addition, being memory efficient is an advantatge.


Application examples:
- An application example for ensemble methods including both the decision tree models and SVM is its implementation for predicting all-cause hospital readmissions of congestive heart failure patients (Artif Intell Med. 2016 Sep;72:72-82. doi: 10.1016/j.artmed.2016.08.005. Epub 2016 Aug 31). A hospital readmission is defined as an admission to a hospital within a certain time frame, typically thirty days, following a previous discharge, either to the same or to a different hospital. Because most patients are not readmitted, the readmission classification problem is highly imbalanced. Here, they used a boosted C5.0 decision tree, as the base classifier, ensembled with a support vector machine (SVM), as a secondary classifier.
- SVM  has also been widely applied to diverse bio-medical fields to address disease subtype identification and pathogenicity of genetic variants. Some citations: Springerplus. 2016 Jul 25;5(1):1162. doi: 10.1186/s40064-016-2677-4. eCollection 2016.
- Decision trees has also been researched for developing intelligent systems for cancer diagnosis and evaluate it on bladder cancer.  They build classification models that are able to discriminate between normal and cancer samples based on the molecular biomarkers discovered. 10.1109/IJCNN.2012.6252781

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [13]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


# TODO: Initialize the three models
clf_A = DecisionTreeClassifier(random_state = 42)
clf_B = RandomForestClassifier(random_state = 42)
clf_C = SVC(random_state = 42)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0026 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.6552.
Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0797 seconds
Made predictions in 0.0133 seconds.
F1 score for training set: 0.9844.
Made predictions in 0.0150 seconds.
F1 score for test set: 0.7153.
Training a SVC using a training set size of 100. . .
Trained model in 0.0018 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.8777.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.7746.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0025 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7500.
Training a RandomForestClassifier using a training 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Decision Tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0026        | 0.0006                 |1                   |0.65             |
| 200               | 0.0025              |  0.0006                      | 1                 |  0.7500               |
| 300               |0.0023                         |0.0003                        |1                  |   0.6613      |

** Classifer 2 - Random Forest**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.0797                       |   0.0133                     |  0.9844                |             0.7153    |
| 200               |    0.0424             |   0.0068                     | 0.99                 |   0.7857              |
| 300               | 0.0655                        |  0.0099                      | 0.9951                 |     0.8092     |

** Classifer 3 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0018                      |  0.0009                      | 0.8777                 |              0.7746   |
| 200               |   0.0037                      | 0.0024                       |  0.8679                |                0.7815 |
| 300               |   0.0069                      | 0.0046                       | 0.8761                 |  0.7838               |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
I would choose as the best model the Random Forest because in the tests done it showed greater generalizability & robustness over the other single estimators:
- It yielded the higher accuracy when predicting student success
- It showed a growing accuracy on the testing set when trained with increasing datasets. 
- Its results, in contrast to the Decision Tree model, where not vulnerable to the different input data used on each training set and the test score showed a continous growth with increasing larger training sets.

On the other hand compared to the other models, this was the most costly in terms of computational resources (training and prediction time). But I would still choose it because the absolute times remain negligible for this data set. If the computational resources were a higher priority for the board of supervisors provided that the training sample would never exceed 10000 samples, then I would choose the Suppor Vector Machine instead. 


### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** The goal of this model, is to predict as much as we can, which are the students that will fail, and those that will pass based on a sample of historical data. The name of this model "Random Forest" is self explanatory of how it works: this is a forest (set) of many different decision trees "randomly or arbitrarily chosen". Each of these decision trees is arbitrarily chosen in order to ensure that they are different from each other and therefore apply a different criteria when deciding if a student will fail or not. Here "arbitrarily chosen" means that each tree has learned to make decisions based on its experience with a limited arbitrary set of students and also they only could judge them based on a limited arbitrary set of characteristics.
Each of these decision trees works as follow: we provide to each tree a set of students defined by their characteristics (age, time of study, etc .. ) together with the graduating outcome (fail or pass) for each student. Then, when we ask this tree wether a particular student will pass or fail, it plays a 20 questions-like game, asking questions like "Number of past class failures?", "Is a boy or a girl?" and so on. The tree asks more informative questions first (i.e., it maximizes the information gain of each question or what is the same it asks first the questions capable of separate best the students that fails or pass based on the possible answers), and gives you a yes/no answer at the end.

Although the particular decisions taken of each single tree are idiosyncratic and non-generalizable because only holds a partial perspective, the final decision taken by the model is the averaged decision for the whole forest, obtaining a highly consetuated decision. In addition the errors that each of the single tree generates get canceled out in the majority. Finally, we  always reserve a sample of the students to measure and know how well it predicts the outcome for students that were not used to train it.  As a result we ends up with a very robust, reproducible and stable model. 


### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [36]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score


# TODO: Create the parameters list you wish to tune
parametersForest = {
    'n_estimators':[10,11,12],
    'max_features': ['sqrt','log2', 31]
    
    
}

# TODO: Initialize the classifier
clf = RandomForestClassifier(random_state = 42)


# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid=parametersForest,  scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0070 seconds.
Tuned model has a training F1 score of 0.9903.
Made predictions in 0.0069 seconds.
Tuned model has a testing F1 score of 0.8116.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
After tuning the parameters, the highest final model's testing F1 score achieved is 0.81 and the training F1 score is almost 1. Although this high training score may suggest overfitting, this seems not to be the case: I observed that the testing score did not deteriorate but converged to a maximum despite that the training scores increased near 1. As expected for this kind of learner that fits a number of decision tree classifiers on various sub-samples of the dataset, it uses averaging to improve the predictive accuracy and control over-fitting.

The untuned model F1 testing score was 0.8092, after tuning it, the final F1 testing score increased to 0.8116 

For tuning I selected the following two main parameters (i.e. the main parameters to consider in Random Forest):  n_estimators and max_features. 
- The n_estimators is the number of trees in the forest. The larger the better, but also the longer it will take to compute. Nonetheless I observed that results stopped getting significantly better beyond a number of 11 trees. 
- The max_features is the size of the random subsets of features to consider when splitting a node. The lower the greater the reduction of variance, but also the greater the increase in bias. Empirical good default values are max_features= sqrt of features. I indeed observed that among the other possibilities this was  one of the best parameter chosen by the GridSearchCV. 

Thus the best parameters chosen after tuning were ({'max_features': 'sqrt', 'n_estimators': 11}) leaving all others to their defaults.

Additional comments beyond the question:
I also tried to tune the model with other parameters like class_weight, max_depth or min_samples_leaf but none of them contributed to increase the final testing F1 score, therefore I disregarded them. Curiosly, for the parameter  class_weight, the GridSearchCV choosed 'balanced' over None, and indeed the mean_test_score was superior for 'balanced' in the GridSearchCV experiment context but decreased the final F1 score compared to class_weight=None. This contradiction can be explained by the fact that the GridSearchCV computed the scores in data sets restricted to the training data, while the final F1 score was computed on a completely different set: the test sample. In theory, because the student sample was inbalanced with more student passing than failing, I would have also included the class_weight to let the GridSearchCV to choose 'balanced' over None but did not included it because decresased the final F1 score. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.